In [4]:
import matplotlib.pyplot as plt
import random
from datetime import datetime, timedelta
import pandas as pd
import json

In [ ]:
def map_to_ocpp_ocpi(data):
    random.seed(42)  # 재현성

    session_id = data.get("sessionID")
    user_id = str(data.get("userID"))
    kwh = round(data.get("kWhDelivered", 0), 3)
    meter_start = random.randint(100000, 300000)
    meter_stop = meter_start + int(kwh * 1000)
    connection_time = data.get("connectionTime")
    disconnect_time = data.get("disconnectTime")
    done_charging = data.get("doneChargingTime")
    user_inputs = data.get("userInputs", [])

    # 상태 결정
    if kwh > 0:
        connector_status = "Charging"
    elif user_inputs:
        connector_status = "Preparing"
    else:
        connector_status = "Available"

    # OCPP 구성
    ocpp_data = {
        "transaction.transactionId": session_id,
        "transaction.idToken.idToken": user_id,
        "transaction.idToken.type": "APP_USER",
        "transaction.meterStart": meter_start,
        "transaction.meterStop": meter_stop,
        "transaction.energyDelivered_kWh": kwh,
        "transaction.chargingStart": connection_time,
        "transaction.chargingEnd": done_charging,
        "transaction.reason": "Local",
        "connector.status": connector_status,
        "connector.voltage": random.choice([220, 230, 380, 400]),
        "connector.current": random.randint(16, 100),
        "connector.power": round(random.uniform(3, 22), 2),
        "connector.temperature": round(random.uniform(35, 85), 1),
        "authorization.status": "Accepted",
        "authorization.certificateStatus": "Valid",
        "diagnostics.errorCode": "NoError",
        "diagnostics.networkStatus": "Online",
        "diagnostics.rssi": random.randint(-85, -40)
    }

    # OCPI 구성
    ocpi_data = {
        "cdr_id": f"CDR_{session_id}",
        "session_id": session_id,
        "auth_id": user_id,
        "token_type": "APP_USER",
        "location_id": data.get("siteID", "LOC001"),
        "evse_uid": data.get("stationID", "EVSE0001"),
        "connector_id": 1,
        "start_date_time": connection_time,
        "end_date_time": disconnect_time,
        "meter_start": meter_start,
        "meter_stop": meter_stop,
        "total_energy": kwh,
        "currency": "KRW",
        "stop_reason": "Local",
        "tariff_id": f"TARIFF_{random.randint(1, 10)}",
        "signed": random.choice([True, False]),
        "last_updated": (datetime.fromisoformat(disconnect_time.replace("Z", "")) + timedelta(minutes=1)).isoformat() + "Z"
    }

    return ocpp_data, ocpi_data


,_id,clusterID,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,sessionID,siteID,spaceID,stationID,timezone,userID,userInputs
0,5dd1f434f9af8b2dec178075,39,2019-11-01T18:11:18Z,2019-11-02T02:55:36Z,2019-11-02T01:16:41Z,14.101,2_39_78_360_2019-11-01 18:11:18.415605,2,CA-322,2-39-78-360,America/Los_Angeles,676.0,"[{'WhPerMile': 400, 'kWhRequested': 24.0, 'mil..."
1,5dd1f434f9af8b2dec178076,39,2019-11-01T18:30:34Z,2019-11-01T22:15:42Z,2019-11-01T22:15:39Z,6.962,2_39_78_367_2019-11-01 18:30:33.896233,2,CA-494,2-39-78-367,America/Los_Angeles,3447.0,"[{'WhPerMile': 375, 'kWhRequested': 15.0, 'mil..."
2,5dd1f434f9af8b2dec178077,39,2019-11-01T18:55:24Z,2019-11-01T19:33:16Z,2019-11-01T19:33:12Z,2.394,2_39_78_361_2019-11-01 18:55:23.831102,2,CA-493,2-39-78-361,America/Los_Angeles,832.0,"[{'WhPerMile': 400, 'kWhRequested': 36.0, 'mil..."
3,5dd1f434f9af8b2dec178078,39,2019-11-01T19:10:30Z,2019-11-02T05:12:26Z,2019-11-01T20:15:26Z,2.160,2_39_123_23_2019-11-01 19:10:29.559896,2,CA-313,2-39-123-23,America/Los_Angeles,1082.0,"[{'WhPerMile': 400, 'kWhRequested': 16.0, 'mil..."
4,5dd1f434f9af8b2dec178079,39,2019-11-01T19:44:41Z,2019-11-01T23:22:16Z,2019-11-01T22:20:49Z,6.756,2_39_78_366_2019-11-01 19:38:12.363403,2,CA-323,2-39-78-366,America/Los_Angeles,1124.0,"[{'WhPerMile': 400, 'kWhRequested': 8.0, 'mile..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6900,61550519f9af8b76960e169a,39,2021-09-13T21:12:53Z,2021-09-14T00:25:36Z,2021-09-13T21:41:31Z,2.285,2_39_89_25_2021-09-13 21:12:53.318460,2,CA-315,2-39-89-25,America/Los_Angeles,431.0,"[{'WhPerMile': 286, 'kWhRequested': 28.6, 'mil..."
6901,61550519f9af8b76960e169b,39,2021-09-13T21:17:04Z,2021-09-14T01:01:49Z,2021-09-13T23:18:07Z,6.715,2_39_123_23_2021-09-13 21:16:44.026068,2,CA-313,2-39-123-23,America/Los_Angeles,6481.0,"[{'WhPerMile': 460, 'kWhRequested': 11.5, 'mil..."
6902,61550519f9af8b76960e169c,39,2021-09-13T22:33:07Z,2021-09-13T23:06:55Z,None,17.720,2_39_81_4550_2021-09-13 22:33:04.543952,2,11900388,2-39-81-4550,America/Los_Angeles,3905.0,"[{'WhPerMile': 300, 'kWhRequested': 30.0, 'mil..."
6903,61550519f9af8b76960e169d,39,2021-09-13T23:11:12Z,2021-09-14T01:43:11Z,2021-09-14T00:13:35Z,2.018,2_39_91_437_2021-09-13 23:10:59.528292,2,CA-317,2-39-91-437,America/Los_Angeles,1082.0,"[{'WhPerMile': 290, 'kWhRequested': 5.8, 'mile..."


In [ ]:
# 고정 seed로 재현성 확보
random.seed(42)

# OCPP 및 OCPI 예시 데이터 생성
ocpp_entries = []
ocpi_entries = []

for i in range(10):
    txn_id = f"TXN{i+1000}"
    id_token = f"TOKEN_{random.randint(100000, 999999)}"
    id_type = random.choice(["ISO14443", "ISO15693", "E_MAID", "APP_USER", "LOCAL"])
    meter_start = random.randint(100000, 500000)
    energy_kwh = round(random.uniform(0.3, 2.5), 2)
    meter_stop = meter_start + int(energy_kwh * 1000)
    charging_start = datetime(2025, 6, 25, 9, 0) + timedelta(minutes=i*15)
    charging_end = charging_start + timedelta(minutes=random.randint(15, 40))
    reason = random.choice([
        "EVDisconnected", "Remote", "DeAuthorized", "EmergencyStop",
        "Timeout", "Local", "EnergyLimitReached", "PowerLimitReached"
    ])
    status = random.choice(["Charging", "Available", "Faulted", "SuspendedEV", "SuspendedEVSE"])
    voltage = random.choice([220, 230, 380, 400, 450])
    current = random.randint(10, 200)
    power = round((voltage * current) / 1000, 2)
    temperature = round(random.uniform(30.0, 95.0), 1)
    auth_status = random.choice(["Accepted", "Blocked", "Expired", "Invalid"])
    cert_status = random.choice(["Valid", "Invalid", "Expired"])
    error_code = random.choice(["NoError", "OverTemperature", "PowerLoss", "GroundFailure", "ConnectorLockFailure"])
    network_status = random.choice(["Online", "Offline", "Unknown"])
    rssi = random.randint(-90, -30)

    ocpp_entries.append({
        "transaction.transactionId": txn_id,
        "transaction.idToken.idToken": id_token,
        "transaction.idToken.type": id_type,
        "transaction.meterStart": meter_start,
        "transaction.meterStop": meter_stop,
        "transaction.energyDelivered_kWh": energy_kwh,
        "transaction.chargingStart": charging_start.isoformat() + "Z",
        "transaction.chargingEnd": charging_end.isoformat() + "Z",
        "transaction.reason": reason,
        "connector.status": status,
        "connector.voltage": voltage,
        "connector.current": current,
        "connector.power": power,
        "connector.temperature": temperature,
        "authorization.status": auth_status,
        "authorization.certificateStatus": cert_status,
        "diagnostics.errorCode": error_code,
        "diagnostics.networkStatus": network_status,
        "diagnostics.rssi": rssi
    })

    ocpi_entries.append({
        "cdr_id": f"CDR_{txn_id}",
        "session_id": txn_id,
        "auth_id": id_token,
        "token_type": "RFID" if "ISO" in id_type else "APP_USER",
        "location_id": f"LOC_{random.randint(100, 999)}",
        "evse_uid": f"EVSE_{random.randint(1000, 9999)}",
        "connector_id": 1,
        "start_date_time": charging_start.isoformat() + "Z",
        "end_date_time": charging_end.isoformat() + "Z",
        "meter_start": meter_start,
        "meter_stop": meter_stop,
        "total_energy": energy_kwh,
        "currency": "KRW",
        "stop_reason": reason,
        "tariff_id": f"TARIFF_{random.randint(1, 10)}",
        "signed": random.choice([True, False]),
        "last_updated": (charging_end + timedelta(minutes=1)).isoformat() + "Z"
    })

ocpp_df = pd.DataFrame(ocpp_entries)
ocpi_df = pd.DataFrame(ocpi_entries)

ocpi_df

In [ ]:
# OCPI 예제 데이터에 이어서 위치(locations), 요금(tariffs), CDR 데이터를 생성

# locations: 충전소 정보
locations = []
tariffs = []
cdrs = []

for i, ocpi_row in enumerate(ocpi_df.itertuples()):
    location_id = ocpi_row.location_id
    evse_uid = ocpi_row.evse_uid
    tariff_id = ocpi_row.tariff_id

    # Location 정보
    locations.append({
        "location_id": location_id,
        "name": f"충전소 {i+1}",
        "evse_uid": evse_uid,
        "address": f"서울특별시 중구 세종대로 {10+i}",
        "city": "서울",
        "postal_code": f"045{i}0",
        "country_code": "KR",
        "latitude": round(37.5665 + i*0.001, 6),
        "longitude": round(126.9780 + i*0.001, 6),
        "type": "PUBLIC"
    })

    # Tariff 정보
    tariffs.append({
        "tariff_id": tariff_id,
        "currency": "KRW",
        "elements": [
            {"price_components": [{"type": "ENERGY", "price": round(250 + i*10, 2), "step_size": 1}]}
        ]
    })

    # CDR 정보
    cdrs.append({
        "cdr_id": ocpi_row.cdr_id,
        "location_id": location_id,
        "evse_uid": evse_uid,
        "connector_id": ocpi_row.connector_id,
        "auth_id": ocpi_row.auth_id,
        "start_date_time": ocpi_row.start_date_time,
        "end_date_time": ocpi_row.end_date_time,
        "meter_start": ocpi_row.meter_start,
        "meter_stop": ocpi_row.meter_stop,
        "total_energy_kWh": ocpi_row.total_energy,
        "currency": ocpi_row.currency,
        "tariff_id": tariff_id,
        "total_cost_KRW": round(ocpi_row.total_energy * (250 + i*10), 2),
        "stop_reason": ocpi_row.stop_reason,
        "last_updated": ocpi_row.last_updated
    })

# 데이터프레임 생성
locations_df = pd.DataFrame(locations)
tariffs_df = pd.DataFrame(tariffs)
cdrs_df = pd.DataFrame(cdrs)

# 파일로 저장
locations_path = "../data/전기차/ocpi_locations.csv"
tariffs_path = "../data/전기차/ocpi_tariffs.csv"
cdrs_path = "../data/전기차/ocpi_cdrs.csv"

locations_df.to_csv(locations_path, index=False)
tariffs_df.to_csv(tariffs_path, index=False)
cdrs_df.to_csv(cdrs_path, index=False)

locations_path, tariffs_path, cdrs_path
